# Containerizer service

Import libraries that have been installed before

In [ ]:
import sklearn
import modzymodel
import mlflow
import joblib
import json

### Train the model

This will train a sklearn model and it will be saved as a joblib file inside the `model` directory.

The goal for containerizer service is to create an image that exposes this model.

In [ ]:
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from joblib import dump, load

digits = datasets.load_digits()
data = digits.images.reshape((len(digits.images), -1))

# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# Learn the digits on the train subset
clf.fit(X_train, y_train)
dump(clf, './model.joblib')

In [ ]:
# Wrap your model in a pyfunc and provide auxiliary functionality through extension of the
# mlflow PythonModel class with methods pre_process, post_process, and explain

class CustomModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        self.model = joblib.load("./model.joblib")

    def predict(self, context, inputs):
        processed_inputs = self.pre_process(inputs)
        inference_results = self.model.predict(processed_inputs)
        return self.post_process(inference_results)

    def pre_process(self, inputs):
        return inputs / 2

    def post_process(self, inference_results):
        structured_results = []
        for inference_result in inference_results:
            inference_result = {
                "classPredictions": [
                    {"class": str(inference_result), "score": str(1)}
                ]
            }
            structured_output = {
                "data": {
                    "result": inference_result,
                    "explanation": None,
                    "drift": None,
                }
            }
            structured_results.append(structured_output)
        return structured_results

    def explain(self, images):
        pass

In [ ]:
# Define conda environment with all required dependencies for your model

conda_env = {
    "channels": ["defaults", "conda-forge", "pytorch"],
    "dependencies": [
        "python=3.8.5",
        "pytorch=1.8.1+cpu",
        "torchvision=0.9.1+cpu",
        "pip",
        {
            "pip": [
                "mlflow",
                "lime",
                "sklearn"
                "cloudpickle==1.6.0",
                "opencv-python"
            ],
        },
    ],
    "name": "linear_env"
}


### Save the model

Transform the model into MLFlow format.

In [ ]:
import mlflow.sklearn
model_save_path = "model/mlflow_custom_pyfunc_svm"
mlflow.pyfunc.save_model(path=model_save_path, python_model=CustomModel(), conda_env=conda_env)

In [ ]:
classifier = mlflow.pyfunc.load_model(model_save_path)
predictions = classifier.predict(X_test)
print(json.dumps(predictions[0], indent=4))

We check that the model has been correctly saved inside the `model` directory.

In [ ]:
!ls ./model/mlflow_custom_pyfunc_svm

### Define the values needed

The fields `name`, `version` and `api_key` are used when uploading the image to Modzy platform.

Since now we are just creating and downloading the docker image, the only fields that containerizer service actually needs are:

* `model_name`: name for the model inside the image
* `model_path`: directory that contains our model file

In [ ]:
api_key = 'XXXX'

image_data = {
    'name': 'docker-registry:5000/mlflow-digits-containerized',
    'version': '0.0.1',
    'model_name': 'digits',
    'model_path': './model/MLFlowModel',
}

### Launch the job

Important fields that we should fill in here are:

* `module`: library that has been used to create the model
* `image_data`: the values defined above
* `image_type`: this is needed in case we are training images so afterwards the proxy will know how to interpret data
* `base_url`: the name of the service that runs the containerizer

In [ ]:
res = modzymodel.publish(
    api_key=api_key,
    module=sklearn,
    image_data=image_data,
    deploy=True,
    image_type=modzymodel.Constants.IMAGE_GREY,
    base_url='http://containerizer:5000'
)

error = res.get('error')
job_id = res.get('job_id')

if error:
    print('Error:', error)
else:
    print('Job ID:', job_id)

After the request is made, containerizer launches a job that runs Kaniko and builds the docker image based on the values provided.

You can get the id of the job created from the result of the request. This id can be used to ask for the status of the job.

In [ ]:
modzymodel.get_job_status(job_id)

Now, we should be able to see the created image listed in the registry. This means that the service has correctly created the image and uploaded it.

In [ ]:
!wget -qO- http://docker-registry:5000/v2/_catalog

### Download the tar file

Now that the job has finished, we can download the docker image that has been generated in the form of a tar file.

In [ ]:
dst = './downloaded_image.tar'
modzymodel.download_tar(job_id, dst)

In [ ]:
!ls -ltr ./

In [ ]:
from IPython.display import display, FileLink

local_file = FileLink(dst, result_html_prefix="Click here to download: ")
display(local_file)
